In [1]:
import numpy as np, pandas as pd, json, glob, os, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
# import shutil

/home/jellybee/anaconda3/envs/llm_311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
